In [1]:
import sys
sys.path.append('..')

In [2]:
from tofina.components import asset, preference
from tofina.theory import varianceMinimization
import tofina.utils as utils
from tofina.macros import portfolioOptimization, optionPricing
import pandas as pd
import numpy as np
import torch

minVarianceFilePath1 = "minVariancePortfolio1.csv"
covariance1 = torch.tensor([[5, 1], [1, 10]]).float() / 100
mean=0.03


portfolio_ = varianceMinimization.GenerateVarianceMinimizationPortfolio(
    mean, covariance1, torch.tensor([1.0, 1.0]), torch.tensor([1.0, 1.0])
)
optim = portfolioOptimization.optimizeStockPortfolioRiskAverse(
    portfolio_, minVarianceFilePath1
)



../tofina/utils.py:13: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  params[key] = torch.nn.Parameter(torch.tensor(val), requires_grad=False)
  9%|▉         | 88/1000 [00:00<00:03, 267.86it/s]


In [3]:
optim.optimizationResult

{'initial_Stock_Company0weight': 0.5,
 'initial_Stock_Company1weight': 0.5,
 'initial_loss': -0.019090916961431503,
 'final_Stock_Company0weight': 0.6950502991676331,
 'final_Stock_Company1weight': 0.30494970083236694,
 'final_loss': -0.020339922979474068,
 'converged': True}

In [3]:
import pandas as pd
df_path = "/Users/andriylevitskyy/Desktop/tofina/tests/data/options/SPY.csv"
SPY_df = pd.read_csv(df_path)

In [9]:
for i, row in SPY_df.iterrows():
    print(row)
    break

Date                 2023-01-03
Expiry Date          2023-01-03
Call Last                 73.66
Call Bid                  70.86
Call Ask                  71.01
Put Last                   0.01
Put Bid                     0.0
Put Ask                    0.01
Strike Price              310.0
Underlying Price         380.82
Name: 0, dtype: object


In [11]:
row["Strike Price"]

310.0

In [1]:
import sys
sys.path.append('..')

In [2]:
from tofina.components.backtest import Backtester
import tofina.components.instrument as instrument
from tofina.extern.arch import forecastDecoratorGARCH
from tofina.extern.yfinance import loadHistoricalStockData
import datetime as dt
import numpy as np
from pathlib import Path
import shutil
import pandas as pd

SPY = loadHistoricalStockData("SPY")
start_date = dt.datetime(2023, 1, 1)
end_date = dt.datetime(2023, 1, 14)
split_date = start_date - dt.timedelta(days=1)
timestamps = SPY.index[
    np.logical_and(SPY.index >= start_date, SPY.index < end_date)
]
horizon = 20
simulations = 1000
arch_kwargs = dict(vol="Garch", p=3, o=0, q=3, dist="t", mean="AR", lags=5)
forecaster = forecastDecoratorGARCH(
    SPY, split_date, horizon, simulations, **arch_kwargs
)
backtester = Backtester(timestamps=timestamps)
backtester.stockDataFromDataFrame(SPY, "SPY")
backtester.registerForecaster(forecaster, ["SPY"])
backtester.addDeposit(interestRate=0.2/252)
df_path = "/Users/andriylevitskyy/Desktop/tofina/tests/data/options/SPY.csv"
df = pd.read_csv(df_path)
df["Date"] = df["Date"].apply(lambda x: x.replace(" ", ""))
df["Expiry Date"] = df["Expiry Date"].apply(lambda x: x.replace(" ", ""))
df["Date"] = pd.to_datetime(df["Date"])
df["Expiry Date"] = pd.to_datetime(df["Expiry Date"])
df = df[df["Date"] >= start_date]
df = df[df["Date"] < end_date]
backtester.parseOptionData(df, "SPY")

[*********************100%%**********************]  1 of 1 completed


Iteration:      5,   Func. Count:     94,   Neg. LLF: 30356.489405585766
Iteration:     10,   Func. Count:    181,   Neg. LLF: 15244.176184581413
Iteration:     15,   Func. Count:    263,   Neg. LLF: 10144.559858087046
Iteration:     20,   Func. Count:    344,   Neg. LLF: 10075.081632452344
Iteration:     25,   Func. Count:    420,   Neg. LLF: 10073.148971941075
Optimization terminated successfully    (Exit mode 0)
            Current function value: 10073.14896934825
            Iterations: 27
            Function evaluations: 449
            Gradient evaluations: 27


/opt/anaconda3/lib/python3.8/site-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alternatively,
you can use the import comment

from arch.__future__ import reindexing

to globally set reindex to True and silence this warning.

  warnings.warn(
34744it [00:57, 607.05it/s] 


In [3]:
dates = list(backtester.pointInTimePortfolio.keys())
for i in range(9):
    print(i,backtester.pointInTimePortfolio[dates[i]].num_instruments)

0 2937
1 2821
2 2617
3 2749
4 2709
5 2863
6 3077
7 2919
8 3055
